<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#시작하며" data-toc-modified-id="시작하며-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>시작하며</a></span></li><li><span><a href="#데이터-전처리와--EDA" data-toc-modified-id="데이터-전처리와--EDA-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>데이터 전처리와  EDA</a></span><ul class="toc-item"><li><span><a href="#데이터-준비" data-toc-modified-id="데이터-준비-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>데이터 준비</a></span></li><li><span><a href="#데이터-탐색" data-toc-modified-id="데이터-탐색-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>데이터 탐색</a></span></li><li><span><a href="#모델-검증을-위한-사용자-초기-정보-세팅" data-toc-modified-id="모델-검증을-위한-사용자-초기-정보-세팅-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>모델 검증을 위한 사용자 초기 정보 세팅</a></span></li><li><span><a href="#모델에-활용하기-위한-전처리-(실습)" data-toc-modified-id="모델에-활용하기-위한-전처리-(실습)-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>모델에 활용하기 위한 전처리 (실습)</a></span></li></ul></li><li><span><a href="#사용자의-명시적/암묵적-평가" data-toc-modified-id="사용자의-명시적/암묵적-평가-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>사용자의 명시적/암묵적 평가</a></span></li><li><span><a href="#CSR(Compressed-Sparse-Row)-Matrix" data-toc-modified-id="CSR(Compressed-Sparse-Row)-Matrix-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>CSR(Compressed Sparse Row) Matrix</a></span></li><li><span><a href="#MF-모델-학습하기" data-toc-modified-id="MF-모델-학습하기-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>MF 모델 학습하기</a></span></li><li><span><a href="#비슷한-아티스트-찾기-+-유저에게-추천하기" data-toc-modified-id="비슷한-아티스트-찾기-+-유저에게-추천하기-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>비슷한 아티스트 찾기 + 유저에게 추천하기</a></span><ul class="toc-item"><li><span><a href="#비슷한-아티스트-찾기" data-toc-modified-id="비슷한-아티스트-찾기-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>비슷한 아티스트 찾기</a></span></li><li><span><a href="#유저에게-아티스트-추천하기" data-toc-modified-id="유저에게-아티스트-추천하기-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>유저에게 아티스트 추천하기</a></span></li></ul></li><li><span><a href="#마무리" data-toc-modified-id="마무리-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>마무리</a></span></li></ul></div>

## 시작하며

0. 추천시스템의 개념과 목적을 이해한다.
1. Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 본다.
2. 음악 감상 기록을 활용하여 비슷한 아티스트를 찾고 아티스트를 추천해 본다.
3. 추천 시스템에서 자주 사용되는 데이터 구조인 CSR Matrix을 익힌다
4. 유저의 행위 데이터 중 Explicit data와 Implicit data의 차이점을 익힌다.
5. 새로운 데이터셋으로 직접 추천 모델을 만들어 본다.

## 데이터 전처리와  EDA

### 데이터 준비

In [1]:
import os
import pandas as pd

fname = os.getenv('HOME') + '/aiffel/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'
col_names = ['user_id', 'artist_MBID', 'artist', 'play']   # 임의로 지정한 컬럼명
data = pd.read_csv(fname, sep='\t', names= col_names)      # sep='\t'로 주어야 tsv를 열 수 있습니다.  
data.head(10)

,user_id,artist_MBID,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,0639533a-0402-40ba-b6e0-18b067198b73,lunachicks,403


In [2]:
# 필요없는 컬럼 제거
using_cols = ['user_id', 'artist', 'play']
data = data[using_cols]
data.head()

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706


In [3]:
# 다른 방법으로 컬럼 제거하기
# del data['artist_MBID']
# data.head()

In [4]:
# 검색을 쉽게하기 위해서 아티스트 문자열을 소문자로 바꿔주기
data['artist'] = data['artist'].str.lower()
data.head()

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706


In [5]:
# 데이터 확인하기
# 첫 번째 유저가 어떤 아티스트의 노래를 듣는지 확인하기
data[data['user_id'] == data.loc[0, 'user_id']]

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17535655 entries, 0 to 17535654
Data columns (total 3 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  object
 1   artist   object
 2   play     int64 
dtypes: int64(1), object(2)
memory usage: 401.4+ MB


In [7]:
data.shape

(17535655, 3)

### 데이터 탐색

+ EDA
    - 유저수, 아티스트수, 인기 많은 아티스트
    - 유저들이 몇 명의 아티스트를 듣고 있는지에 대한 통계
    - 유저 play 횟수 중앙값에 대한 통계

> pandas.DataFrame.nunique()

In [8]:
# 유저 수
data['user_id'].nunique()

358868

In [9]:
# 아티스트 수
data['artist'].nunique()

291346

In [10]:
# 인기많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head()

artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
Name: user_id, dtype: int64

In [11]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

### 모델 검증을 위한 사용자 초기 정보 세팅

In [12]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['black eyed peas' , 'maroon5' ,'jason mraz' ,'coldplay' ,'beyoncé']

# 'mingki'라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['mingki']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['mingki']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,mingki,black eyed peas,30
1,mingki,maroon5,30
2,mingki,jason mraz,30
3,mingki,coldplay,30
4,mingki,beyoncé,30


In [13]:
data.isin({'user_id':['mingki']})['user_id'].any()

True

### 모델에 활용하기 위한 전처리 (실습)

우리가 다루는 데이터에서는 user와 artist 각각에 번호를 붙이고 싶습니다. 보통 이런 작업을 indexing이라고 합니다.

> pandas.DataFrame.unique()

In [14]:
# 고유한 유저, 아티스트로 배열 만들어주기
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

In [15]:
{j:i for i, j in enumerate(user_unique[:5])}

{'00000c289a1829a808ac09c00daf10bc3c4e223b': 0,
 '00001411dc427966b17297bf4d69e7e193135d89': 1,
 '00004d2ac9316e22dc007ab2243d6fcb239e707d': 2,
 '000063d3fe1cf2ba248b9e3c3f0334845a27a6bf': 3,
 '00007a47085b9aab8af55f52ec8846ac479ac4fe': 4}

In [16]:
# 인덱싱
user_to_idx = {j:i for i, j in enumerate(user_unique)}
artist_to_idx = {j:i for i, j in enumerate(artist_unique)}

In [17]:
# 인덱싱이 잘 되었는지 확인하기
print(user_to_idx['mingki'])
print(artist_to_idx['maroon5'])

358868
270115


In [18]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해보자.
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될테니 dropna()로 제거한다.
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
temp_user_data

0         0
1         0
2         0
3         0
4         0
      ...  
0    358868
1    358868
2    358868
3    358868
4    358868
Name: user_id, Length: 17535660, dtype: int64

In [19]:
data['user_id'].map(user_to_idx.get).dropna()

0         0
1         0
2         0
3         0
4         0
      ...  
0    358868
1    358868
2    358868
3    358868
4    358868
Name: user_id, Length: 17535660, dtype: int64

In [20]:
if len(temp_user_data)==len(data): # 모든 row가 정상적으로 인덱싱 되었다면
    print('user_id column indexing OK!!')
    data['user_id']=temp_user_data # data['user_id']을 인덱싱된 Series로 교체
else:
    print('user_id column indexing Fail!!')

user_id column indexing OK!!


In [21]:
# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data

artist column indexing OK!!


,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
...,...,...,...
0,358868,376,30
1,358868,270115,30
2,358868,3746,30
3,358868,62,30


## 사용자의 명시적/암묵적 평가

+ 명시적(explicit)
    - 좋아요, 별점
+ 암묵적(implicit)

In [25]:
# 1회만 play한 데이터의 비율을 보는 코드
only_one = data[data['play']<2]

one, all_data = len(only_one), len(data)
print(one, all_data, '\n')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')

147740 17535660 

Ratio of only_one over all data is 0.84%


우리가 앞으로 만들어갈 모델에서는 암묵적 데이터의 해석을 위해 다음과 같은 규칙을 적용할 것입니다.

1. 한 번이라도 들었으면 선호한다고 판단한다.
2. 많이 재생한 아티스트에 대해 가중치를 주어서 더 확실히 좋아한다고 판단한다.

> Matrix Factorization(MF)

## CSR(Compressed Sparse Row) Matrix

```python
csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])  
where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k]., M,N은 matrix의 shape
```

In [29]:
# CSR Matrix 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()
print(num_user, num_artist, '\n')

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape=(num_user, num_artist))
print(csr_data)

358869 291347 

  (0, 0)	2137
  (0, 1)	1099
  (0, 2)	897
  (0, 3)	717
  (0, 4)	706
  (0, 5)	691
  (0, 6)	545
  (0, 7)	507
  (0, 8)	424
  (0, 9)	403
  (0, 10)	393
  (0, 11)	387
  (0, 12)	361
  (0, 13)	358
  (0, 14)	329
  (0, 15)	316
  (0, 16)	310
  (0, 17)	302
  (0, 18)	288
  (0, 19)	281
  (0, 20)	244
  (0, 21)	232
  (0, 22)	231
  (0, 23)	229
  (0, 24)	227
  :	:
  (358867, 4542)	23
  (358867, 4896)	12
  (358867, 4947)	19
  (358867, 5395)	14
  (358867, 5562)	39
  (358867, 5858)	93
  (358867, 5867)	16
  (358867, 6855)	36
  (358867, 7856)	46
  (358867, 9641)	45
  (358867, 11203)	40
  (358867, 11631)	12
  (358867, 15227)	51
  (358867, 21303)	14
  (358867, 25966)	12
  (358867, 28563)	12
  (358867, 49416)	54
  (358867, 73333)	14
  (358867, 97322)	15
  (358867, 140606)	11
  (358868, 62)	30
  (358868, 376)	30
  (358868, 396)	30
  (358868, 3746)	30
  (358868, 270115)	30


## MF 모델 학습하기

Matrix Factorization 모델을 implicit 패키지를 사용하여 학습해 봅시다.

+ implicit 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지입니다.
+ 이 패키지에 구현된 als(AlternatingLeastSquares) 모델을 사용하겠습니다. Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 것으로 알려져 있습니다.

In [30]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

AlternatingLeastSquares 클래스의 \_\_init\_\_ 파라미터를 살펴보겠습니다.

1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 
3. use_gpu : GPU를 사용할 것인지 
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

1,4를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 합니다.

In [31]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [32]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<291347x358869 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Column format>

In [33]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

모델이 

1. mingki의 벡터와 black eyed peas의 벡터를 어떻게 만들고 있는지와 
2. 두 벡터를 곱하면 어떤 값이 나오는지 확인해보자

In [34]:
mingki, black_eyed_peas = user_to_idx['mingki'], artist_to_idx['black eyed peas']
mingki_vector, black_eyed_peas_vector = als_model.user_factors[mingki], als_model.item_factors[black_eyed_peas]

In [35]:
mingki_vector

array([-3.83735567e-01, -1.23676217e+00, -1.40806392e-01,  3.40656340e-01,
       -1.09270155e+00,  4.54874933e-02, -1.36691064e-01,  5.95259011e-01,
       -6.02212608e-01, -4.17082906e-01, -3.34860533e-02, -4.18518446e-02,
        7.07242012e-01,  5.66836953e-01,  1.68607748e+00, -6.25044465e-01,
        3.53346825e-01, -5.26049554e-01,  4.64677244e-01, -8.85980017e-03,
       -5.61957806e-02,  1.10386632e-01,  4.22694057e-01, -2.34800600e-03,
       -8.10281277e-01,  1.32103503e+00, -4.90551740e-01, -1.03418267e+00,
        1.36612117e+00,  7.45039061e-02,  3.94948483e-01, -5.16591728e-01,
        7.13973939e-01, -3.04117203e-01,  3.99797291e-01,  7.18800426e-01,
        2.69561201e-01,  4.99989063e-01, -2.67915912e-02,  3.41405720e-01,
       -2.56065875e-01,  2.33823568e-01,  2.88126171e-02, -4.27389145e-01,
       -5.78258097e-01, -9.47586954e-01, -1.68849647e-01,  5.93479753e-01,
       -4.19676542e-01, -5.00210404e-01, -2.43659690e-01, -1.56476557e-01,
       -1.52068615e+00, -

In [36]:
black_eyed_peas_vector

array([ 0.00250109,  0.00208457, -0.01711112, -0.01254841, -0.01168851,
        0.0046178 ,  0.00610716,  0.00332234,  0.00203546, -0.00915099,
        0.00958754,  0.00674012,  0.01236502,  0.00980316,  0.02194051,
        0.00113018,  0.01182256,  0.00488442,  0.00870972,  0.00552423,
       -0.00121838,  0.00120894,  0.01600437,  0.00358528, -0.00152492,
        0.01611694, -0.00154885, -0.01523089,  0.01753458, -0.00041767,
        0.02372853, -0.00119951,  0.01540025,  0.00313716,  0.01574198,
        0.00447259,  0.01175291,  0.02306204,  0.01537561,  0.01057643,
        0.00322336, -0.00156966,  0.00929087, -0.00220727,  0.01141554,
        0.00750667,  0.01157629,  0.01453355, -0.00282058,  0.01500169,
        0.00153026,  0.01130919, -0.00734093, -0.0084358 , -0.0021203 ,
        0.02044763,  0.03440222,  0.01743187,  0.00630796,  0.01200541,
        0.00750066,  0.01301515,  0.01801098,  0.00539395, -0.0023731 ,
        0.01871411,  0.00538497,  0.02651937,  0.0104226 ,  0.00

In [37]:
# mingki와 black_eyed_peas를 내적하는 코드
np.dot(mingki_vector, black_eyed_peas_vector)

0.48400587

In [39]:
# 나의 Queen에 대한 선호도를 어떻게 예측하는지 확인하기
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(mingki_vector, queen_vector)

0.28803676

In [41]:
# 확인
artist_to_idx['queen']

75

## 비슷한 아티스트 찾기 + 유저에게 추천하기

### 비슷한 아티스트 찾기

---

AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 비슷한 아티스트를 찾는다.

In [42]:
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)
similar_artist

[(62, 0.99999994),
 (277, 0.9883233),
 (5, 0.981684),
 (28, 0.9792143),
 (217, 0.9701359),
 (473, 0.96990097),
 (418, 0.96952754),
 (247, 0.9652551),
 (490, 0.96283054),
 (694, 0.9561561),
 (910, 0.9517499),
 (782, 0.9490773),
 (1018, 0.9473282),
 (268, 0.946832),
 (75, 0.934987)]

(아티스트의 id, 유사도) Tuple로 반환.

In [43]:
# artist_to_idx를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성하기
idx_to_artist = {v:k for k, v in artist_to_idx.items()}
idx_to_artist

{0: 'betty blowtorch',
 1: 'die ärzte',
 2: 'melissa etheridge',
 3: 'elvenking',
 4: 'juliette & the licks',
 5: 'red hot chili peppers',
 6: 'magica',
 7: 'the black dahlia murder',
 8: 'the murmurs',
 9: 'lunachicks',
 10: 'walls of jericho',
 11: 'letzte instanz',
 12: 'goldfrapp',
 13: 'horrorpops',
 14: 'the butchies',
 15: 'jack off jill',
 16: 'babes in toyland',
 17: 'dropkick murphys',
 18: 'all:my:faults',
 19: 'le tigre',
 20: 'schandmaul',
 21: 'edguy',
 22: 'maximum the hormone',
 23: 'all ends',
 24: 'jack johnson',
 25: 'eluveitie',
 26: 'rasputina',
 27: 'london after midnight',
 28: 'the killers',
 29: 'mutyumu',
 30: 'judas priest',
 31: 'rob zombie',
 32: 'the bosshoss',
 33: 'blue öyster cult',
 34: 'sandra nasic',
 35: 'john mayer',
 36: 'sleater-kinney',
 37: 'the who',
 38: 'disciple',
 39: 'tanzwut',
 40: 'guano apes',
 41: 'the rolling stones',
 42: 'little big town',
 43: 'team dresch',
 44: 'betty',
 45: 'l7',
 46: 'bif naked',
 47: 'girlschool',
 48: 'the w

In [45]:
[idx_to_artist[i[0]] for i in similar_artist]

['coldplay',
 'muse',
 'red hot chili peppers',
 'the killers',
 'radiohead',
 'placebo',
 'u2',
 'the beatles',
 'oasis',
 'foo fighters',
 'nirvana',
 'the white stripes',
 'the smashing pumpkins',
 'pink floyd',
 'queen']

In [46]:
# 코드 함수로 만들어주기
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

print("완료오")

완료오


In [47]:
get_similar_artist('2pac')

['2pac',
 'nas',
 'notorious b.i.g.',
 'the game',
 'dr. dre',
 'jay-z',
 'ludacris',
 'ice cube',
 '50 cent',
 'snoop dogg']

In [48]:
get_similar_artist('lady gaga')

['lady gaga',
 'britney spears',
 'katy perry',
 'rihanna',
 'beyoncé',
 'the pussycat dolls',
 'kelly clarkson',
 'christina aguilera',
 'pink',
 'leona lewis']

### 유저에게 아티스트 추천하기

---


AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 제가 좋아할 만한 아티스트를 추천받는다.

filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument이다.

In [49]:
user = user_to_idx['mingki']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

[(350, 0.4649889),
 (1800, 0.4521076),
 (550, 0.44828832),
 (369, 0.44559696),
 (2249, 0.44404596),
 (354, 0.41937998),
 (274, 0.41510072),
 (627, 0.40544838),
 (391, 0.3997802),
 (355, 0.39843953),
 (409, 0.38578185),
 (621, 0.3847586),
 (5555, 0.38201624),
 (724, 0.38064545),
 (901, 0.37948197),
 (24, 0.37323323),
 (618, 0.37225682),
 (358, 0.3674389),
 (1777, 0.36323345),
 (564, 0.3448195)]

In [50]:
[idx_to_artist[i[0]] for i in artist_recommended]

['rihanna',
 'lady gaga',
 'britney spears',
 'justin timberlake',
 'katy perry',
 'nelly furtado',
 'michael jackson',
 'maroon 5',
 'christina aguilera',
 'madonna',
 'amy winehouse',
 'alicia keys',
 'timbaland',
 'lily allen',
 'pink',
 'jack johnson',
 'the pussycat dolls',
 'kelly clarkson',
 'mariah carey',
 'kanye west']

rihanna를 추천해주고 있네요. 모델은 왜 rihanna를 추천해줬을까? 

AlternatingLeastSquares 클래스에 구현된 explain 메소드를 사용하면 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인할 수 있다.

In [51]:
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)

In [54]:
print(len(explain))
explain

3


(0.46619557636353576,
 [(396, 0.20933865521868983),
  (376, 0.15184489538140572),
  (3746, 0.061451087964159115),
  (62, 0.043327786867204385),
  (270115, 0.0002331509320767174)],
 (array([[ 2.89597294e+00,  2.24469103e+00,  2.58387849e+00, ...,
           2.77025396e+00,  2.39734896e+00,  2.39880382e+00],
         [ 6.50056449e+00,  1.20530257e+00,  5.33223931e-01, ...,
           5.04270540e-01,  5.12363080e-01,  4.01765445e-01],
         [ 7.48284220e+00,  6.44270505e+00,  1.08721853e+00, ...,
           4.15530262e-01,  3.47800143e-01,  3.60242130e-01],
         ...,
         [ 8.02258051e+00,  6.82616280e+00,  7.87866095e+00, ...,
           8.64808077e-01, -6.13671689e-03,  3.20468405e-02],
         [ 6.94265771e+00,  5.99886024e+00,  6.84579743e+00, ...,
           7.36876889e+00,  8.47237366e-01,  2.36369350e-02],
         [ 6.94687097e+00,  5.86882235e+00,  6.80411048e+00, ...,
           7.39594250e+00,  6.40156218e+00,  8.62267968e-01]]),
  False))

이 method는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수)를 반환. 

어떤 아티스트들이 이 추천에 얼마나 기여하고 있는 걸까 확인할 수 있다.

In [53]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]

[('beyoncé', 0.20933865521868983),
 ('black eyed peas', 0.15184489538140572),
 ('jason mraz', 0.061451087964159115),
 ('coldplay', 0.043327786867204385),
 ('maroon5', 0.0002331509320767174)]

## 마무리

---

1. 유저, 아티스트에 대한 Meta 정보를 반영하기 쉽지 않다. 연령대별로 음악 취향이 굉장히 다르지 않을까?
2. 유저가 언제 play 했는지에 반영하기 쉽지 않다. 10년 전에 즐겨듣던 아티스트와 지금 즐겨듣는 아티스트를 비교해보자.